# Easy Problem

We want to minimize the total cost of deliveries!

In [154]:
import numpy as np
import matplotlib as plt

## Parameters

We have the following problem parameters:
- Cost of the DC in euros $C = 25000$
- Maximum number of DCs $N = 20$
- Expected deliveries per 1000 people $D = 0.2$
- Cost per kilometer of each delivery $K = 1$
- Radius of the Earth $R = 6371.009$

In [155]:
PARAM = {
    "C": 25000,
    "N": 20,
    "D": 0.2 / 1000,
    "K": 1,
    "R": 6371.009
}

# Model

We can then construct the model, where the decision variables are binary and correspond to the answer to the question "Will there be a distribution center in this town?".

Let the towns be represented by a number $i$, then $x_i$ is $1$ if there is a DC in $i$ and $0$ otherwise.

We can then model the problem as follows:
$$
\begin{alignat*}{2}

\min_{x_i} &\quad& &K \times \text{Sum of the distance from each city to the closest DC} + C \times \sum_i x_i
\\\\

\text{s.t}& &&x_i \in \{0, 1\}
\\\\
&&& \sum_i x_i \ge 1
\\
&&& \sum_i x_i \le N

\end{alignat*}
$$

## Data

Open the data we'll work with.

In [156]:
data = np.loadtxt("PopulationCountPT-2020.csv", delimiter=",", dtype=object)

index, name, pop, lat, lon = (data[1:, i] for i in range(5))

pop = pop.astype("float64")
lat = lat.astype("float64")
lon = lon.astype("float64")

# Calculate expected number of deliveries
val = pop * PARAM["D"]
VAL = val


# Convert the lats and lons to radians
lat = lat * np.pi / 180
lon = lon * np.pi / 180


# Number of cities
NC = int(index[-1])
print(NC)

379


## Organize the Data

We'll organize the data that relates to each city into a dict and store the distance between any pair of cities in a matrix.

In [157]:
def create_city(id: int, name: str, val: int, lat: float, lon: float) -> dict:
    return {
        "id": id,
        "name": name,
        "val": val,
        "lat": lat,
        "lon": lon
    }


def dist(city1: dict, city2: dict) -> float:
    global PARAM
    R = PARAM["R"]

    theta = min(city1["lat"], city2["lat"])

    dtheta = abs(city1["lat"] - city2["lat"])
    dphi = abs(city1["lon"] - city2["lon"])

    return R * np.sin(theta) * dphi + R * dtheta
    


# List with all the cities
CC = []
for id, n, v, la, lo in zip(index, name, val, lat, lon):
    CC += [create_city(int(id)-1, n, v, la, lo)]


DD = np.zeros((NC, NC))
for i1, city1 in enumerate(CC):
    for i2, city2 in enumerate(CC[i1:]):
        DD[i1, i1 + i2] = dist(city1, city2)

DD = DD + DD.transpose()
print(DD[0:2, 0:7])

[[  0.         306.85927257  10.91193996 364.69597021  38.35659918
  215.20893961  12.86465043]
 [306.85927257   0.         309.54427733  58.52795115 311.10935063
  118.70640199 310.88942488]]


# Solving the Problem

We are going to use the following "greedy" heuristic for solving the problem:
- Calculate the best place to put each DC and put it there!

To implement it, we are going to do the following:
- Compute the distances between every pair of cities and order them from closest to farthest.
- Calculate the value that we get from putting a DC in each available city, supposing each city is supplied by the closest DC to it, and choose the highest.

## Preliminaries

In [158]:
class NumberBase:
    def __init__(self, base: int, dims: int) -> None:
        self.base = base
        self.dims = dims

        self.value = [0 for _ in range(dims)]
    

    def __str__(self) -> str:
        return f"{self.value}"
    

    def increment_pos_n(self, n) -> list[int]:
        self.value[n] += 1
        if self.value[n] >= self.base:
            if n != 0:
                self.value[n] = 0
                return self.increment_pos_n(n-1)
            else:
                raise ValueError("Overflow!")
        
        return self.value
        

    
    def increment(self) -> list[int]:
        return self.increment_pos_n(self.dims - 1)
        



def eval_DCs(DD: np.ndarray, VAL: np.ndarray,  DCs: list[int], len_DCs: int) -> float:
    """
        Given an array of DCs return their total cost
    """
    global PARAM

    min_dists = np.min(DD[:, DCs], axis=1)
    cost = np.sum(min_dists * VAL)
    cost += PARAM["C"] * (len_DCs + 1)

    return cost


def new_DC(NC: int, DD: np.ndarray, VAL: np.ndarray,  DCs: list[int], cost: float) -> tuple[float, float]:
    """
        Given the number of cities NC and the currente DCs return the best new DC and the respective cost
    """

    min_cost = cost
    choice = -1
    len_DCs = len(DCs)

    options = np.array(range(NC))
    options = np.delete(options, DCs)

    for new_DC in options:
        all_DCs = DCs.copy() + [new_DC]

        cost = eval_DCs(DD, VAL, all_DCs, len_DCs + 1)

        if cost < min_cost:
            min_cost = cost
            choice = new_DC
    
    return choice, min_cost


def new_multi_DC(NC: int, DD: np.ndarray, VAL: np.ndarray,  DCs: list[int], cost: float, nDC: int) -> tuple[float, float]:
    """
        Given the number of cities NC and the currente DCs return the best new nDC number of DCs and the respective total cost
    """

    min_cost = cost
    choices = np.array([-1 for _ in range(nDC)])
    len_DCs = len(DCs)

    options = np.array(range(NC))
    options = np.delete(options, DCs)

    i = NumberBase(NC, nDC)
    for _ in range(NC**nDC - 1):
        new_DCs = i.value.copy()

        # Prevent permutations
        if nDC > 1:
            cont = False
            for index in range(nDC - 1):
                if new_DCs[index + 1] <= new_DCs[index]:
                    cont = True
                    break
            if cont:
                i.increment()
                continue


        all_DCs = DCs.copy() + new_DCs

        cost = eval_DCs(DD, VAL, all_DCs, len_DCs + nDC)

        if cost < min_cost:
            min_cost = cost
            choices = new_DCs
        
        i.increment()
            
    
    return choices, min_cost

## Upper Bound

We can set an easy upper bound and sanity check by simply placing DCs on the 20 highest populated cities and seeing the result:

In [159]:
DCs = range(20)

cost = eval_DCs(DD, VAL, DCs, len(DCs))

print(f"Total Cost: {cost/1e6:.2f} million euros")
print(f"Number of DCs: {len(DCs)}")
print(DCs)

Total Cost: 0.56 million euros
Number of DCs: 20
range(0, 20)


## Greedy Heuristic

Now let's implement our greedy heuristic:

In [160]:
DCs = []
cost = np.inf
for _ in range(PARAM["N"]):
    choice, cost = new_DC(NC, DD, VAL, DCs, cost)
    if choice == -1:
        continue

    DCs += [choice]

print(f"Total Cost: {cost/1e3:.2f} thousand euros")
print(f"Number of DCs: {len(DCs)}")
print(DCs)

Total Cost: 0.17 thousand euros
Number of DCs: 3
[49, 70, 0]


## Multi-Greedy Heuristic

We can now place multiple DCs at the same time!

In [161]:
DCs = []
cost = np.inf

for _ in range(PARAM["N"]):
    choice, cost = new_multi_DC(NC, DD, VAL, DCs, cost, 1)
    if np.sum(choice) < 0:
        continue

    DCs += choice

cost = eval_DCs(DD, VAL, DCs, len(DCs))
print(f"Total Cost: {cost/1e3:.2f} thousand euros")
print(f"Number of DCs: {len(DCs)}")
print(DCs)

Total Cost: 169.68 thousand euros
Number of DCs: 3
[49, 70, 0]


In [162]:
DCs = []
cost = np.inf

for _ in range(PARAM["N"]//2):
    choice, cost = new_multi_DC(NC, DD, VAL, DCs, cost, 2)
    if np.sum(choice) < 0:
        continue

    DCs += choice

cost = eval_DCs(DD, VAL, DCs, len(DCs))
print(f"Total Cost: {cost/1e3:.2f} thousand euros")
print(f"Number of DCs: {len(DCs)}")
print(DCs)

Total Cost: 0.15 thousand euros
Number of DCs: 2
[0, 70]


In [163]:
DCs = []
cost = np.inf

for _ in range(PARAM["N"]//4):
    choice, cost = new_multi_DC(NC, DD, VAL, DCs, cost, 4)
    if np.sum(choice) < 0:
        continue

    DCs += choice

cost = eval_DCs(DD, VAL, DCs, len(DCs))
print(f"Total Cost: {cost/1e3:.2f} thousand euros")
print(f"Number of DCs: {len(DCs)}")
print(DCs)

KeyboardInterrupt: 